# Image Transformations Walkthrough

Let's start by making an example image:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# I define this image via an outer product: 
#   (sin(x)^2) . (y^2)
base_image = np.outer(np.arange(16)**2,
                      np.sin(np.arange(41)/4)**2) 

def plot_image(image, title=None):
    plt.figure(dpi=150)
    plt.imshow(base_image, cmap='gray')
    if title is not None:
        plt.title(title)
        
plot_image(base_image, title='Our Base Image')
print(f"Image Shape: {base_image.shape}")
None

**Question:** What is the (x, y) coordinate of the top left corner?

**Question:** What is the (x, y) coordinate of the bottom right corner?

Remember that we want to represent a transformation matrix as follows:

$$ \begin{bmatrix}x' \\ y'\end{bmatrix} = 
T \begin{bmatrix}x \\ y\end{bmatrix} $$

What we want to do is to build a data structure (a matrix) that contains all of the image coordinates and allows us to see what applying a `transformation_matrix` $T$ will do to them.

Let's write them out here:

In [ ]:
## TASK: Write out the 2D image coordinates for the four corners
#  as a single matrix. Each coordinate should be a column vector.

# Note: writing a matrix of column vectors is annoying. It is far
# easier to write out a matrix of row vectors and then take the
# transpose operation of that matrix.
initial_image_coords = None

if initial_image_coords is None:
    raise NotImplementedError('You need to define the image coordinates')
if not initial_image_coords.shape == (2, 4):
    raise ValueError('You should have 4 2-element column vectors.')
    
print(f'Image Coordinates: \n'
      f'  x: {initial_image_coords[0]}\n'
      f'  y: {initial_image_coords[1]}')

Once we're done defining the coordinates, we can plot them on top of the image.

In [ ]:
# Once we're done, we can plot them.
def plot_coords(coordinates, fmt='bo'):
    plt.plot(coordinates[0, :], coordinates[1, :], fmt)

plot_image(base_image, 'Base Image with Coords')
plot_coords(initial_image_coords)

Recall that we want to represent a transformation matrix as follows:

$$ \begin{bmatrix}x' \\ y'\end{bmatrix} = 
T \begin{bmatrix}x \\ y\end{bmatrix}$$

Now that we have a matrix of the image coordinates (each with their own $x$ and $y$), we can go about defining some transformation matrices. Let's start with the identity matrix.

**Question:** What is the shape of this transformation matrix?

**Question:** What are the entries in the identity transformation?

**TASK:** Let's implement the identity transformation matrix.

In [ ]:
s = 2
transformation = None


if transformation is None:
    raise NotImplementedError()


transformed_image_coords = transformation @ initial_image_coords

plot_image(base_image)
plot_coords(initial_image_coords, 'bo')
plot_coords(transformed_image_coords, 'rx')

Now let's think about some other transformation matrices:

- [ ] Rotation
- [ ] Skew
- [ ] Inverse Transformations (easy with `np.linalg.inv`!)

## Homogeneous Image Coordinates

In order to support translation, we need to extend our coordinates from above to include a third coordinate, turning them into *homogeneous image coordinates*. 

**Task:** Let's add a third coordinate to all of our coordinates from before. (What value does this third coordinate have?)

**Question:** What does the identity matrix look like in this new system?

**Task:** Let's implement translation.

**Task:** Let's implement a *homography* (Don't forget to re-homogonize the coordinates!)

In [ ]:
## TODO: Let's implement the homogeneous image coordinates.

initial_image_coords = None


transformation = None

if transformation is None:
    raise NotImplementedError()

transformed_image_coords = transformation @ initial_image_coords

print(f"Before:\n {initial_image_coords}")

print(f"After:\n {transformed_image_coords}")

plot_image(base_image)
plot_coords(initial_image_coords, 'bo')
plot_coords(transformed_image_coords, 'rx')

Okay, lets plot those coordinates too (and on top of the original image).

## Image Warping: Starting Point Code

Here, I am providing you with code that you can use to get started on implementing full image warping. I have modified the `upsample_image` code slightly to use the `scipy.interpolate` package: you **can** use this for your second programming assignment.

In this first code block, I have re-implemented the image upsampling procedure that you should recognize from your homework:

In [ ]:
## Upsampling image example
from scipy.interpolate import RectBivariateSpline

def upsample_image(image, target_shape):
    # Initialize the upsampled image
    image_up = np.zeros(target_shape)
    
    sh = image.shape
    x = np.arange(image.shape[1]).astype(float)
    y = np.arange(image.shape[0]).astype(float)
    
    # Define the new coordinates (using the [y, x] convention
    # since image matrices are defined [row, column])
    new_xs = np.linspace(0, image.shape[1]-1, 
                         target_shape[1], endpoint=True)
    new_ys = np.linspace(0, image.shape[0]-1, 
                         target_shape[0], endpoint=True)
    
    # Loop through coordinates and set the image values
    # Note: you can use this 'image_fn' to perform interpolation instead
    # of your implementation: new_val = image_fn(new_x, new_y)[0, 0]
    image_fn = RectBivariateSpline(x, y, image.T)  # Transpose needed for proper x, y coordinate
    for ix, new_x in np.ndenumerate(new_xs):
        for iy, new_y in np.ndenumerate(new_ys):
            image_up[iy, ix] = image_fn(new_x, new_y)
            
    return image_up


up_image = upsample_image(base_image, [16*3, 41*3])

plt.figure(dpi=300)
plt.subplot(2, 1, 1)
plt.imshow(base_image, cmap='gray')
plt.subplot(2, 1, 2)
plt.imshow(up_image, cmap='gray')

None

In your homework, you will need to use what you have learned today in class to implement image warping. You can use the image upsampling + interpolation code above as a starting point. To warp an image, you will need to loop over all pixels in the warped target image and:

1. Compute their location in the input image
2. Get the pixel value from the input image using interpolation
3. Set the value of the target-image pixel with that value.

The code will look something like this:

In [ ]:
def transform_image(image, transformation_matrix):
    # Notice that because matrices are stored "rows, columns",
    # we need to flip the "shape" coordinates so that the transformation
    # matrix does what we expect. The other convention is also acceptable,
    # as long as one is consistent. In this function, the transformation
    # matrix is assumed to be in [x, y, w] coordinates, even though the image
    # is stored in row, column (y, x) coordinates.
    sh = image.shape
    x = np.arange(image.shape[1]).astype(float)
    y = np.arange(image.shape[0]).astype(float)
    
    # For now, the dimensions of the output image will
    # remain unchanged. Modify xi and yi to change the
    # domain of the output image.
    xi = np.arange(image.shape[1]).astype(float)
    yi = np.arange(image.shape[0]).astype(float)

    # Perform the transformation
    image_fn = scipy.interpolate.interp2d(x, y, image, fill_value=0)
    transformed_image = np.zeros((len(yi), len(xi)))
    raise NotImplementedError()
    return transformed_image